# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
cd '../gdrive/MyDrive/SSAC/3조'

/gdrive/.shortcut-targets-by-id/15_BxZVEQYCdGCGiQ5nexpWPc1cgHVe4w/3조


# DeepFM

In [ ]:
import deepctr_torch

In [ ]:
# !pip install -U deepctr-torch

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *

# 파일 불러오기

- 이미지 벡터 없는 파일

In [ ]:
df = pd.read_csv('data/total_data.csv')
df.head(2)

,content_id,imp_id,view_time,bid_price,user_id,label,imp_time,adv_id,content_name,content_price,content_used,content_cat_1,content_cat_2,content_cat_3,content_emergency_count,content_comment_count,content_views,content_likes,content_b_pay,content_delivery_fee,content_status,content_img_url,adv_follower_count,adv_grade,adv_item_count,adv_views,adv_review_count,adv_comment_count,adv_pay_count,adv_parcel_post_count,adv_transfer_count,adv_chat_count,viewer_gender,viewer_age,viewer_following_count,viewer_pay_count,viewer_parcel_post_count,viewer_transfer_count,viewer_chat_count,viewer_age_ch,content_place,b_pay_rate,sec,title_len
0,137078818,99d1612dded122458e99,16:56:46,50,1728109,1,16:48:41,12851816,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,250000,1,전자제품,600700,600700001,1,4,25583,817,0,배송비포함,삭제됨,https://media.bunjang.co.kr/product/137078818_...,2131,5285,247,4525,542,739,496,0,0,11,1,30,5,0,0,0,0,30,서울특별시,0.394224,484.0,34
1,137078818,99d1612d825321055790,00:00:00,50,10733469,1,17:00:45,12851816,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,250000,1,전자제품,600700,600700001,1,4,25583,817,0,배송비포함,삭제됨,https://media.bunjang.co.kr/product/137078818_...,2131,5285,247,4525,542,739,496,0,0,11,1,18,0,0,0,0,0,18,서울특별시,0.394224,7.0,34


- 이미지 벡터 포함한 파일

In [ ]:
df['content_place'].value_counts()

전국        337762
경기도       151537
서울특별시     145182
대구광역시      36163
부산광역시      23272
인천광역시      16085
대전광역시      15283
충청북도        4177
전라북도        2632
경상북도        2389
경상남도        1212
Name: content_place, dtype: int64

In [ ]:
df = pd.read_csv('data/total_data_img.csv', low_memory=False)

# 준비

- 평가함수 정의

In [ ]:
from sklearn.metrics import log_loss, roc_auc_score

grouped_label = df.groupby('label').size()
average_ctr = float(grouped_label[1]/grouped_label.sum())
average_ctr

def get_rig(train_y, test_y, pred):
    avg_ctr = average_ctr
    prior = log_loss(train_y, [avg_ctr]*len(train_y))

    classifier = log_loss(test_y, pred)

    rig = (prior - classifier) / prior
    return rig


# 전처리

### 학습에 사용할 feature 분류

In [ ]:
categorical = [
    'viewer_gender',
    'content_used',
    'content_cat_1',
    'content_cat_2',
    'content_cat_3',
    "content_status",
    'content_delivery_fee',
    'content_b_pay', 
    'content_place'
    ]
continuous = [
    'bid_price', 
    'content_price',
    # 'content_emergency_count',
    'content_comment_count',     
    'content_views',       
    'content_likes',
    'adv_follower_count',
    # 'adv_grade',       
    'adv_item_count',         
    'adv_views',
    'adv_review_count',
    'adv_comment_count',
    'adv_pay_count',
    'adv_parcel_post_count', 
    'adv_transfer_count', 
    # 'adv_chat_count',
    'viewer_age',
    'viewer_following_count',
    'viewer_pay_count',
    "viewer_parcel_post_count",
    'viewer_transfer_count',
    'viewer_chat_count', 
    # 'b_pay_rate',
    # 'viewer_age_ch',
    # 'sec', 
    'title_len'
    ]

# 이미지 벡터 사용하는 경우
vec_features = [str(i) for i in range(512)]

### 결측치

In [ ]:
# 결측치 채우기
df[categorical] = df[categorical].fillna('-1', )
df[continuous] = df[continuous].fillna(0, )
target = ['label']

### Label Encoding, Scaling

In [ ]:
# # categorical 데이터는 Label Encoding
for feat in categorical:
    lbe = LabelEncoder()
    df[feat] = lbe.fit_transform(df[feat])

# categorical 데이터는 One-hot Encoding
# df = pd.get_dummies(df, columns=categorical, drop_first=True)

# continuous 데이터는 Min-Max Scaling
# scaler = MinMaxScaler(feature_range=(0, 1))

# continuous 데이터는 Standard Scaling
scaler = StandardScaler()
df[continuous] = scaler.fit_transform(df[continuous])


In [ ]:
categorical = df.columns[35:].tolist()

In [ ]:
df.columns[34:]

Index(['title_len', 'viewer_gender_1', 'viewer_gender_2', 'content_used_1',
       'content_cat_1_가방', 'content_cat_1_기타', 'content_cat_1_남성 악세사리',
       'content_cat_1_남성 의류', 'content_cat_1_도서/티켓/문구', 'content_cat_1_미용품',
       ...
       'content_place_경상남도', 'content_place_경상북도', 'content_place_대구광역시',
       'content_place_대전광역시', 'content_place_부산광역시 ', 'content_place_서울특별시',
       'content_place_인천광역시', 'content_place_전국', 'content_place_전라북도',
       'content_place_충청북도'],
      dtype='object', length=316)

In [ ]:
scaler = StandardScaler()
df[vec_features] = scaler.fit_transform(df[vec_features])



In [ ]:
continuous = continuous + vec_features

In [ ]:
len(df.columns)

44

### train test split

In [ ]:
# 3.generate input data for model

train, test = train_test_split(df, test_size=0.2, random_state=47)#, stratify=df['label'])

### 각 feature 정보를 형식에 맞춰 저장

In [ ]:
# 2.count #unique features for each sparse field,and record dense feature field name

# 각 feature마다 값의 종류가 몇개인지, feature 이름이 무엇인지 등 저장
fixlen_feature_columns = [SparseFeat(feat, df[feat].nunique())
                            for feat in categorical] + [DenseFeat(feat, 1, )
                                                            for feat in continuous]

# dnn feature에 일단 모든 feature 정보 넣기
dnn_feature_columns = fixlen_feature_columns

# lenear feature에도 일단 모든 feature 정보 넣기
linear_feature_columns = fixlen_feature_columns 

# feature 이름들만 따로 저장
feature_names = get_feature_names(
    linear_feature_columns + dnn_feature_columns)

In [ ]:
# train/test 데이터의 각 feature에 대응하는 컬럼을 딕셔너리로 저장
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

## 파라미터 목록

- DeepFM(linear_feature_columns, dnn_feature_columns, use_fm=True, dnn_hidden_units=(256, 128), l2_reg_linear=1e-05, l2_reg_embedding=1e-05, l2_reg_dnn=0, init_std=0.0001, seed=1024, dnn_dropout=0, dnn_activation='relu', dnn_use_bn=False, task='binary', device='cpu', gpus=None)
### Parameters:	
- linear_feature_columns – An iterable containing all the features used by linear part of the model.
- dnn_feature_columns – An iterable containing all the features used by deep part of the model.
- use_fm – bool,use FM part or not
- dnn_hidden_units – list,list of positive integer or empty list, the layer - - number and units in each layer of DNN
- l2_reg_linear – float. L2 regularizer strength applied to linear part
- l2_reg_embedding – float. L2 regularizer strength applied to embedding vector
- l2_reg_dnn – float. L2 regularizer strength applied to DNN
- init_std – float,to use as the initialize std of embedding vector
- seed – integer ,to use as random seed.
- dnn_dropout – float in [0,1), the probability we will drop out a given DNN coordinate.
- dnn_activation – Activation function to use in DNN
- dnn_use_bn – bool. Whether use BatchNormalization before activation or not in DNN
- task – str, "binary" for binary logloss or "regression" for regression loss
- device – str, "cpu" or "cuda:0"
- gpus – list of int or torch.device for multiple gpus. If None, run on device. gpus[0] should be the same gpu with device.

In [ ]:
# linear_feature_columns – An iterable containing all the features used by linear part of the model.
# dnn_feature_columns – An iterable containing all the features used by deep part of the model.
use_fm – True / False
# dnn_hidden_units – list,list of positive integer or empty list, the layer - - number and units in each layer of DNN
# l2_reg_linear – float. L2 regularizer strength applied to linear part
# l2_reg_embedding – float. L2 regularizer strength applied to embedding vector
# l2_reg_dnn – float. L2 regularizer strength applied to DNN
# init_std – float,to use as the initialize std of embedding vector
seed – integer ,to use as random seed.
dnn_dropout – 0~1
dnn_activation – 'relu', 'tanh'
dnn_use_bn – True / False
task – "binary"
device – "cpu" or "cuda:0"
gpus – list of int or torch.device for multiple gpus. If None, run on device. gpus[0] should be the same gpu with device.

# 학습

In [ ]:
!nvidia-smi

Mon Dec 27 02:22:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    32W / 250W |   1169MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 크롤링 불가능한 3개 feature 제외

## One hot

- one-hot encoding, `adv_grade`, `adv_chat`, `content_emergency` 제외
- category 1, 2, 3

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='relu', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/20
229s - loss:  0.1378 - binary_crossentropy:  0.1378 - auc:  0.6844 - val_binary_crossentropy:  0.1335 - val_auc:  0.6935
Epoch 2/20
227s - loss:  0.1353 - binary_crossentropy:  0.1353 - auc:  0.6987 - val_binary_crossentropy:  0.1315 - val_auc:  0.7050
Epoch 3/20
228s - loss:  0.1338 - binary_crossentropy:  0.1338 - auc:  0.7115 - val_binary_crossentropy:  0.1312 - val_auc:  0.7056
Epoch 4/20
225s - loss:  0.1334 - binary_crossentropy:  0.1334 - auc:  0.7178 - val_binary_crossentropy:  0.1318 - val_auc:  0.7072
Epoch 5/20
224s - loss:  0.1329 - binary_crossentropy:  0.1329 - auc:  0.7218 - val_binary_crossentropy:  0.1303 - val_auc:  0.7136
Epoch 6/20
227s - loss:  0.1326 - binary_crossentropy:  0.1327 - auc:  0.7259 - val_binary_crossentropy:  0.1314 - val_auc:  0.7148
Epoch 7/20
228s - loss:  0.1318 - binary_crossentropy:  0.1318 - auc:  0.7302 - val_b

- one-hot encoding, `adv_grade`, `adv_chat`, `content_emergency` 제외
- category 2

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='relu', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/20
61s - loss:  0.1381 - binary_crossentropy:  0.1381 - auc:  0.6856 - val_binary_crossentropy:  0.1319 - val_auc:  0.6974
Epoch 2/20
60s - loss:  0.1345 - binary_crossentropy:  0.1345 - auc:  0.7033 - val_binary_crossentropy:  0.1305 - val_auc:  0.7101
Epoch 3/20
59s - loss:  0.1329 - binary_crossentropy:  0.1329 - auc:  0.7183 - val_binary_crossentropy:  0.1303 - val_auc:  0.7137
Epoch 4/20
60s - loss:  0.1322 - binary_crossentropy:  0.1323 - auc:  0.7259 - val_binary_crossentropy:  0.1297 - val_auc:  0.7197
Epoch 5/20
59s - loss:  0.1314 - binary_crossentropy:  0.1314 - auc:  0.7317 - val_binary_crossentropy:  0.1294 - val_auc:  0.7218
Epoch 6/20
59s - loss:  0.1308 - binary_crossentropy:  0.1308 - auc:  0.7350 - val_binary_crossentropy:  0.1294 - val_auc:  0.7197
Epoch 7/20
60s - loss:  0.1302 - binary_crossentropy:  0.1302 - auc:  0.7402 - val_binary_c

- one-hot encoding, `adv_grade`, `adv_chat`, `content_emergency` 제외
- category 1

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='relu', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/20
27s - loss:  0.1390 - binary_crossentropy:  0.1390 - auc:  0.6781 - val_binary_crossentropy:  0.1322 - val_auc:  0.6926
Epoch 2/20
26s - loss:  0.1350 - binary_crossentropy:  0.1350 - auc:  0.6968 - val_binary_crossentropy:  0.1317 - val_auc:  0.7002
Epoch 3/20
26s - loss:  0.1337 - binary_crossentropy:  0.1337 - auc:  0.7101 - val_binary_crossentropy:  0.1306 - val_auc:  0.7123
Epoch 4/20
26s - loss:  0.1328 - binary_crossentropy:  0.1327 - auc:  0.7178 - val_binary_crossentropy:  0.1301 - val_auc:  0.7154
Epoch 5/20
26s - loss:  0.1322 - binary_crossentropy:  0.1322 - auc:  0.7234 - val_binary_crossentropy:  0.1298 - val_auc:  0.7174
Epoch 6/20
26s - loss:  0.1316 - binary_crossentropy:  0.1316 - auc:  0.7283 - val_binary_crossentropy:  0.1292 - val_auc:  0.7236
Epoch 7/20
26s - loss:  0.1310 - binary_crossentropy:  0.1310 - auc:  0.7325 - val_binary_c

## Label

- Label encoding, `adv_grade`, `adv_chat`, `content_emergency` 제외
- Category 1

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='relu', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/20
12s - loss:  0.1393 - binary_crossentropy:  0.1393 - auc:  0.6771 - val_binary_crossentropy:  0.1324 - val_auc:  0.6884
Epoch 2/20
11s - loss:  0.1350 - binary_crossentropy:  0.1350 - auc:  0.6976 - val_binary_crossentropy:  0.1327 - val_auc:  0.6952
Epoch 3/20
11s - loss:  0.1340 - binary_crossentropy:  0.1340 - auc:  0.7067 - val_binary_crossentropy:  0.1320 - val_auc:  0.6986
Epoch 4/20
11s - loss:  0.1334 - binary_crossentropy:  0.1334 - auc:  0.7128 - val_binary_crossentropy:  0.1312 - val_auc:  0.7064
Epoch 5/20
11s - loss:  0.1329 - binary_crossentropy:  0.1329 - auc:  0.7182 - val_binary_crossentropy:  0.1309 - val_auc:  0.7093
Epoch 6/20
12s - loss:  0.1323 - binary_crossentropy:  0.1323 - auc:  0.7222 - val_binary_crossentropy:  0.1302 - val_auc:  0.7164
Epoch 7/20
11s - loss:  0.1317 - binary_crossentropy:  0.1317 - auc:  0.7272 - val_binary_c

- Label encoding, `adv_grade`, `adv_chat`, `content_emergency` 제외
- Category 2

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='relu', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/20
12s - loss:  0.1400 - binary_crossentropy:  0.1401 - auc:  0.6746 - val_binary_crossentropy:  0.1323 - val_auc:  0.6942
Epoch 2/20
11s - loss:  0.1345 - binary_crossentropy:  0.1345 - auc:  0.7026 - val_binary_crossentropy:  0.1311 - val_auc:  0.7099
Epoch 3/20
11s - loss:  0.1336 - binary_crossentropy:  0.1336 - auc:  0.7128 - val_binary_crossentropy:  0.1325 - val_auc:  0.7056
Epoch 4/20
11s - loss:  0.1329 - binary_crossentropy:  0.1329 - auc:  0.7167 - val_binary_crossentropy:  0.1321 - val_auc:  0.6985
Epoch 5/20
11s - loss:  0.1324 - binary_crossentropy:  0.1323 - auc:  0.7236 - val_binary_crossentropy:  0.1296 - val_auc:  0.7220
Epoch 6/20
11s - loss:  0.1317 - binary_crossentropy:  0.1317 - auc:  0.7286 - val_binary_crossentropy:  0.1304 - val_auc:  0.7142
Epoch 7/20
11s - loss:  0.1311 - binary_crossentropy:  0.1311 - auc:  0.7320 - val_binary_c

- Label encoding, `adv_grade`, `adv_chat`, `content_emergency` 제외
- Category 3

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='relu', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/20
11s - loss:  0.1413 - binary_crossentropy:  0.1413 - auc:  0.6797 - val_binary_crossentropy:  0.1335 - val_auc:  0.6897
Epoch 2/20
11s - loss:  0.1344 - binary_crossentropy:  0.1344 - auc:  0.7042 - val_binary_crossentropy:  0.1322 - val_auc:  0.6992
Epoch 3/20
11s - loss:  0.1334 - binary_crossentropy:  0.1334 - auc:  0.7137 - val_binary_crossentropy:  0.1312 - val_auc:  0.7120
Epoch 4/20
11s - loss:  0.1326 - binary_crossentropy:  0.1326 - auc:  0.7224 - val_binary_crossentropy:  0.1314 - val_auc:  0.7177
Epoch 5/20
11s - loss:  0.1319 - binary_crossentropy:  0.1319 - auc:  0.7279 - val_binary_crossentropy:  0.1297 - val_auc:  0.7204
Epoch 6/20
11s - loss:  0.1313 - binary_crossentropy:  0.1313 - auc:  0.7310 - val_binary_crossentropy:  0.1299 - val_auc:  0.7208
Epoch 7/20
11s - loss:  0.1307 - binary_crossentropy:  0.1307 - auc:  0.7365 - val_binary_c

- Label encoding, `adv_grade`, `adv_chat`, `content_emergency` 제외
- Category 1,2,3

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='relu', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/20
13s - loss:  0.1411 - binary_crossentropy:  0.1411 - auc:  0.6809 - val_binary_crossentropy:  0.1316 - val_auc:  0.7040
Epoch 2/20
12s - loss:  0.1342 - binary_crossentropy:  0.1341 - auc:  0.7086 - val_binary_crossentropy:  0.1313 - val_auc:  0.7040
Epoch 3/20
12s - loss:  0.1330 - binary_crossentropy:  0.1330 - auc:  0.7185 - val_binary_crossentropy:  0.1311 - val_auc:  0.7118
Epoch 4/20
12s - loss:  0.1322 - binary_crossentropy:  0.1322 - auc:  0.7245 - val_binary_crossentropy:  0.1305 - val_auc:  0.7153
Epoch 5/20
12s - loss:  0.1316 - binary_crossentropy:  0.1317 - auc:  0.7296 - val_binary_crossentropy:  0.1315 - val_auc:  0.7021
Epoch 6/20
12s - loss:  0.1309 - binary_crossentropy:  0.1309 - auc:  0.7350 - val_binary_crossentropy:  0.1310 - val_auc:  0.7198
Epoch 7/20
12s - loss:  0.1304 - binary_crossentropy:  0.1304 - auc:  0.7387 - val_binary_c

# Activation Function 비교

기존 ReLU : `test LogLoss 0.1311`, `test AUC 0.7339`, `test RIG 0.074`  
Mish : `test LogLoss 0.1311`, `test AUC 0.7402`, `test RIG 0.0737`  
Swish : `test LogLoss 0.1309`, `test AUC 0.7379`, `test RIG 0.075`  
Tanh  
Sigmoid  
Linear  
PReLU  

## Mish

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='mish', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
Mish activation
Mish activation
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/20
17s - loss:  0.1487 - binary_crossentropy:  0.1480 - auc:  0.6744 - val_binary_crossentropy:  0.1326 - val_auc:  0.6850
Epoch 2/20
17s - loss:  0.1347 - binary_crossentropy:  0.1347 - auc:  0.6994 - val_binary_crossentropy:  0.1317 - val_auc:  0.6992
Epoch 3/20
17s - loss:  0.1338 - binary_crossentropy:  0.1338 - auc:  0.7080 - val_binary_crossentropy:  0.1316 - val_auc:  0.7031
Epoch 4/20
17s - loss:  0.1334 - binary_crossentropy:  0.1334 - auc:  0.7120 - val_binary_crossentropy:  0.1305 - val_auc:  0.7098
Epoch 5/20
16s - loss:  0.1324 - binary_crossentropy:  0.1324 - auc:  0.7213 - val_binary_crossentropy:  0.1300 - val_auc:  0.7187
Epoch 6/20
17s - loss:  0.1319 - binary_crossentropy:  0.1319 - auc:  0.7272 - val_binary_crossentropy:  0.1298 - val_auc:  0.7199
Epoch 7/20
17s - loss:  0.1312 - binary_crossentropy:  0.13

## Swish

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='swish', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/20
13s - loss:  0.1513 - binary_crossentropy:  0.1502 - auc:  0.6731 - val_binary_crossentropy:  0.1326 - val_auc:  0.6863
Epoch 2/20
13s - loss:  0.1348 - binary_crossentropy:  0.1348 - auc:  0.6987 - val_binary_crossentropy:  0.1319 - val_auc:  0.6972
Epoch 3/20
12s - loss:  0.1338 - binary_crossentropy:  0.1338 - auc:  0.7087 - val_binary_crossentropy:  0.1311 - val_auc:  0.7067
Epoch 4/20
12s - loss:  0.1333 - binary_crossentropy:  0.1333 - auc:  0.7136 - val_binary_crossentropy:  0.1305 - val_auc:  0.7095
Epoch 5/20
12s - loss:  0.1324 - binary_crossentropy:  0.1323 - auc:  0.7217 - val_binary_crossentropy:  0.1300 - val_auc:  0.7188
Epoch 6/20
12s - loss:  0.1318 - binary_crossentropy:  0.1318 - auc:  0.7277 - val_binary_crossentropy:  0.1296 - val_auc:  0.7221
Epoch 7/20
12s - loss:  0.1312 - binary_crossentropy:  0.1311 - auc:  0.7321 - val_binary_c

## tanh

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='tanh', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/20
12s - loss:  0.1833 - binary_crossentropy:  0.1833 - auc:  0.6371 - val_binary_crossentropy:  0.1329 - val_auc:  0.6812
Epoch 2/20
12s - loss:  0.1348 - binary_crossentropy:  0.1348 - auc:  0.6976 - val_binary_crossentropy:  0.1315 - val_auc:  0.6987
Epoch 3/20
12s - loss:  0.1337 - binary_crossentropy:  0.1337 - auc:  0.7089 - val_binary_crossentropy:  0.1307 - val_auc:  0.7087
Epoch 4/20
12s - loss:  0.1329 - binary_crossentropy:  0.1329 - auc:  0.7156 - val_binary_crossentropy:  0.1299 - val_auc:  0.7170
Epoch 5/20
12s - loss:  0.1321 - binary_crossentropy:  0.1321 - auc:  0.7228 - val_binary_crossentropy:  0.1301 - val_auc:  0.7183
Epoch 6/20
12s - loss:  0.1315 - binary_crossentropy:  0.1315 - auc:  0.7285 - val_binary_crossentropy:  0.1294 - val_auc:  0.7217
Epoch 7/20
12s - loss:  0.1310 - binary_crossentropy:  0.1310 - auc:  0.7324 - val_binary_c

## Sigmoid

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='sigmoid', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/20
12s - loss:  0.1450 - binary_crossentropy:  0.1449 - auc:  0.6710 - val_binary_crossentropy:  0.1328 - val_auc:  0.6793
Epoch 2/20
12s - loss:  0.1356 - binary_crossentropy:  0.1356 - auc:  0.6878 - val_binary_crossentropy:  0.1324 - val_auc:  0.6858
Epoch 3/20
12s - loss:  0.1349 - binary_crossentropy:  0.1349 - auc:  0.6938 - val_binary_crossentropy:  0.1319 - val_auc:  0.6932
Epoch 4/20
12s - loss:  0.1343 - binary_crossentropy:  0.1343 - auc:  0.7003 - val_binary_crossentropy:  0.1314 - val_auc:  0.7013
Epoch 5/20
15s - loss:  0.1338 - binary_crossentropy:  0.1338 - auc:  0.7050 - val_binary_crossentropy:  0.1309 - val_auc:  0.7069
Epoch 6/20
14s - loss:  0.1334 - binary_crossentropy:  0.1334 - auc:  0.7116 - val_binary_crossentropy:  0.1306 - val_auc:  0.7095
Epoch 7/20
14s - loss:  0.1329 - binary_crossentropy:  0.1329 - auc:  0.7152 - val_binary_c

## Linear

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='linear', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/20
12s - loss:  0.2098 - binary_crossentropy:  0.1991 - auc:  0.6487 - val_binary_crossentropy:  0.1346 - val_auc:  0.6616
Epoch 2/20
12s - loss:  0.1363 - binary_crossentropy:  0.1363 - auc:  0.6818 - val_binary_crossentropy:  0.1331 - val_auc:  0.6821
Epoch 3/20
12s - loss:  0.1358 - binary_crossentropy:  0.1358 - auc:  0.6834 - val_binary_crossentropy:  0.1331 - val_auc:  0.6806
Epoch 4/20
12s - loss:  0.1354 - binary_crossentropy:  0.1354 - auc:  0.6869 - val_binary_crossentropy:  0.1326 - val_auc:  0.6850
Epoch 5/20
12s - loss:  0.1352 - binary_crossentropy:  0.1352 - auc:  0.6864 - val_binary_crossentropy:  0.1326 - val_auc:  0.6888
Epoch 6/20
12s - loss:  0.1351 - binary_crossentropy:  0.1351 - auc:  0.6902 - val_binary_crossentropy:  0.1325 - val_auc:  0.6858
Epoch 7/20
12s - loss:  0.1348 - binary_crossentropy:  0.1348 - auc:  0.6933 - val_binary_c

## PReLU

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='prelu', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/20
12s - loss:  0.1436 - binary_crossentropy:  0.1436 - auc:  0.6802 - val_binary_crossentropy:  0.1320 - val_auc:  0.6931
Epoch 2/20
12s - loss:  0.1344 - binary_crossentropy:  0.1344 - auc:  0.7045 - val_binary_crossentropy:  0.1313 - val_auc:  0.7046
Epoch 3/20
12s - loss:  0.1335 - binary_crossentropy:  0.1335 - auc:  0.7129 - val_binary_crossentropy:  0.1312 - val_auc:  0.7082
Epoch 4/20
12s - loss:  0.1328 - binary_crossentropy:  0.1328 - auc:  0.7179 - val_binary_crossentropy:  0.1300 - val_auc:  0.7167
Epoch 5/20
12s - loss:  0.1319 - binary_crossentropy:  0.1319 - auc:  0.7257 - val_binary_crossentropy:  0.1298 - val_auc:  0.7202
Epoch 6/20
12s - loss:  0.1314 - binary_crossentropy:  0.1314 - auc:  0.7315 - val_binary_crossentropy:  0.1297 - val_auc:  0.7198
Epoch 7/20
12s - loss:  0.1308 - binary_crossentropy:  0.1308 - auc:  0.7359 - val_binary_c

# Epoch 50

## Mish

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='mish', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=50, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/50
12s - loss:  0.1487 - binary_crossentropy:  0.1480 - auc:  0.6744 - val_binary_crossentropy:  0.1326 - val_auc:  0.6850
Epoch 2/50
12s - loss:  0.1347 - binary_crossentropy:  0.1347 - auc:  0.6993 - val_binary_crossentropy:  0.1317 - val_auc:  0.6987
Epoch 3/50
12s - loss:  0.1339 - binary_crossentropy:  0.1339 - auc:  0.7079 - val_binary_crossentropy:  0.1314 - val_auc:  0.7035
Epoch 4/50
12s - loss:  0.1334 - binary_crossentropy:  0.1334 - auc:  0.7122 - val_binary_crossentropy:  0.1305 - val_auc:  0.7099
Epoch 5/50
12s - loss:  0.1324 - binary_crossentropy:  0.1324 - auc:  0.7211 - val_binary_crossentropy:  0.1299 - val_auc:  0.7192
Epoch 6/50
12s - loss:  0.1319 - binary_crossentropy:  0.1319 - auc:  0.7272 - val_binary_crossentropy:  0.1298 - val_auc:  0.7200
Epoch 7/50
12s - loss:  0.1311 - binary_crossentropy:  0.1311 - auc:  0.7317 - val_binary_c

## Swish

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='swish', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=50, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/50
12s - loss:  0.1513 - binary_crossentropy:  0.1502 - auc:  0.6731 - val_binary_crossentropy:  0.1326 - val_auc:  0.6863
Epoch 2/50
12s - loss:  0.1348 - binary_crossentropy:  0.1348 - auc:  0.6987 - val_binary_crossentropy:  0.1319 - val_auc:  0.6972
Epoch 3/50
12s - loss:  0.1338 - binary_crossentropy:  0.1338 - auc:  0.7087 - val_binary_crossentropy:  0.1311 - val_auc:  0.7067
Epoch 4/50
12s - loss:  0.1333 - binary_crossentropy:  0.1333 - auc:  0.7136 - val_binary_crossentropy:  0.1305 - val_auc:  0.7095
Epoch 5/50
12s - loss:  0.1324 - binary_crossentropy:  0.1323 - auc:  0.7217 - val_binary_crossentropy:  0.1300 - val_auc:  0.7188
Epoch 6/50
12s - loss:  0.1318 - binary_crossentropy:  0.1318 - auc:  0.7277 - val_binary_crossentropy:  0.1296 - val_auc:  0.7221
Epoch 7/50
12s - loss:  0.1312 - binary_crossentropy:  0.1311 - auc:  0.7321 - val_binary_c

## Tanh

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='tanh', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=50, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/50
12s - loss:  0.1833 - binary_crossentropy:  0.1833 - auc:  0.6371 - val_binary_crossentropy:  0.1329 - val_auc:  0.6812
Epoch 2/50
12s - loss:  0.1348 - binary_crossentropy:  0.1348 - auc:  0.6976 - val_binary_crossentropy:  0.1315 - val_auc:  0.6987
Epoch 3/50
12s - loss:  0.1337 - binary_crossentropy:  0.1337 - auc:  0.7089 - val_binary_crossentropy:  0.1307 - val_auc:  0.7087
Epoch 4/50
12s - loss:  0.1329 - binary_crossentropy:  0.1329 - auc:  0.7156 - val_binary_crossentropy:  0.1299 - val_auc:  0.7170
Epoch 5/50
12s - loss:  0.1321 - binary_crossentropy:  0.1321 - auc:  0.7228 - val_binary_crossentropy:  0.1301 - val_auc:  0.7183
Epoch 6/50
12s - loss:  0.1315 - binary_crossentropy:  0.1315 - auc:  0.7285 - val_binary_crossentropy:  0.1294 - val_auc:  0.7217
Epoch 7/50
12s - loss:  0.1310 - binary_crossentropy:  0.1310 - auc:  0.7324 - val_binary_c

## ReLU

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='relu', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=50, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/50
12s - loss:  0.1408 - binary_crossentropy:  0.1408 - auc:  0.6813 - val_binary_crossentropy:  0.1321 - val_auc:  0.6948
Epoch 2/50
12s - loss:  0.1341 - binary_crossentropy:  0.1341 - auc:  0.7073 - val_binary_crossentropy:  0.1313 - val_auc:  0.7070
Epoch 3/50
12s - loss:  0.1330 - binary_crossentropy:  0.1331 - auc:  0.7173 - val_binary_crossentropy:  0.1306 - val_auc:  0.7112
Epoch 4/50
12s - loss:  0.1323 - binary_crossentropy:  0.1323 - auc:  0.7226 - val_binary_crossentropy:  0.1295 - val_auc:  0.7210
Epoch 5/50
12s - loss:  0.1314 - binary_crossentropy:  0.1314 - auc:  0.7304 - val_binary_crossentropy:  0.1293 - val_auc:  0.7244
Epoch 6/50
12s - loss:  0.1310 - binary_crossentropy:  0.1310 - auc:  0.7349 - val_binary_crossentropy:  0.1295 - val_auc:  0.7223
Epoch 7/50
12s - loss:  0.1304 - binary_crossentropy:  0.1304 - auc:  0.7390 - val_binary_c

## Sigmoid

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='sigmoid', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=50, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/50
12s - loss:  0.1450 - binary_crossentropy:  0.1449 - auc:  0.6710 - val_binary_crossentropy:  0.1328 - val_auc:  0.6793
Epoch 2/50
12s - loss:  0.1356 - binary_crossentropy:  0.1356 - auc:  0.6878 - val_binary_crossentropy:  0.1324 - val_auc:  0.6858
Epoch 3/50
12s - loss:  0.1349 - binary_crossentropy:  0.1349 - auc:  0.6938 - val_binary_crossentropy:  0.1319 - val_auc:  0.6932
Epoch 4/50
12s - loss:  0.1343 - binary_crossentropy:  0.1343 - auc:  0.7003 - val_binary_crossentropy:  0.1314 - val_auc:  0.7013
Epoch 5/50
12s - loss:  0.1338 - binary_crossentropy:  0.1338 - auc:  0.7050 - val_binary_crossentropy:  0.1309 - val_auc:  0.7069
Epoch 6/50
12s - loss:  0.1334 - binary_crossentropy:  0.1334 - auc:  0.7116 - val_binary_crossentropy:  0.1306 - val_auc:  0.7095
Epoch 7/50
12s - loss:  0.1329 - binary_crossentropy:  0.1329 - auc:  0.7152 - val_binary_c

# Epoch 30

## Linear

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='mish', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=30, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/30
13s - loss:  0.1487 - binary_crossentropy:  0.1480 - auc:  0.6744 - val_binary_crossentropy:  0.1326 - val_auc:  0.6850
Epoch 2/30
12s - loss:  0.1347 - binary_crossentropy:  0.1347 - auc:  0.6993 - val_binary_crossentropy:  0.1317 - val_auc:  0.6987
Epoch 3/30
14s - loss:  0.1339 - binary_crossentropy:  0.1339 - auc:  0.7079 - val_binary_crossentropy:  0.1314 - val_auc:  0.7035
Epoch 4/30
12s - loss:  0.1334 - binary_crossentropy:  0.1334 - auc:  0.7122 - val_binary_crossentropy:  0.1305 - val_auc:  0.7099
Epoch 5/30
12s - loss:  0.1324 - binary_crossentropy:  0.1324 - auc:  0.7211 - val_binary_crossentropy:  0.1299 - val_auc:  0.7192
Epoch 6/30
12s - loss:  0.1319 - binary_crossentropy:  0.1319 - auc:  0.7272 - val_binary_crossentropy:  0.1298 - val_auc:  0.7200
Epoch 7/30
12s - loss:  0.1311 - binary_crossentropy:  0.1311 - auc:  0.7317 - val_binary_c

## PReLU

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='swish', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
hist = model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=30, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/30
12s - loss:  0.1513 - binary_crossentropy:  0.1502 - auc:  0.6731 - val_binary_crossentropy:  0.1326 - val_auc:  0.6863
Epoch 2/30
12s - loss:  0.1348 - binary_crossentropy:  0.1348 - auc:  0.6987 - val_binary_crossentropy:  0.1319 - val_auc:  0.6972
Epoch 3/30
12s - loss:  0.1338 - binary_crossentropy:  0.1338 - auc:  0.7087 - val_binary_crossentropy:  0.1311 - val_auc:  0.7067
Epoch 4/30
12s - loss:  0.1333 - binary_crossentropy:  0.1333 - auc:  0.7136 - val_binary_crossentropy:  0.1305 - val_auc:  0.7095
Epoch 5/30
12s - loss:  0.1324 - binary_crossentropy:  0.1323 - auc:  0.7217 - val_binary_crossentropy:  0.1300 - val_auc:  0.7188
Epoch 6/30
12s - loss:  0.1318 - binary_crossentropy:  0.1318 - auc:  0.7277 - val_binary_crossentropy:  0.1296 - val_auc:  0.7221
Epoch 7/30
12s - loss:  0.1312 - binary_crossentropy:  0.1311 - auc:  0.7321 - val_binary_c

In [ ]:
from deepctr_torch.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_binary_crossentropy', min_delta=0, verbose=1, patience=5, mode='auto')

hist = model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=50, verbose=2, validation_split=0.2, callbacks=[es])

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 256), dnn_dropout=0.0,
               dnn_activation='mish', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )

# 모델 학습
# mdckpt = ModelCheckpoint(filepath='김동영/model.ckpt', monitor='val_binary_crossentropy', verbose=1, save_best_only=True, mode='min')

es = EarlyStopping(monitor='val_binary_crossentropy', min_delta=0, verbose=1, patience=5, mode='auto')
hist = model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=50, verbose=2, validation_split=0.2, callbacks=[es])

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/50
14s - loss:  0.1561 - binary_crossentropy:  0.1534 - auc:  0.6718 - val_binary_crossentropy:  0.1325 - val_auc:  0.6917
Epoch 2/50
13s - loss:  0.1348 - binary_crossentropy:  0.1348 - auc:  0.6974 - val_binary_crossentropy:  0.1324 - val_auc:  0.6935
Epoch 3/50
12s - loss:  0.1338 - binary_crossentropy:  0.1338 - auc:  0.7078 - val_binary_crossentropy:  0.1308 - val_auc:  0.7093
Epoch 4/50
12s - loss:  0.1330 - binary_crossentropy:  0.1330 - auc:  0.7168 - val_binary_crossentropy:  0.1307 - val_auc:  0.7122
Epoch 5/50
12s - loss:  0.1326 - binary_crossentropy:  0.1326 - auc:  0.7203 - val_binary_crossentropy:  0.1300 - val_auc:  0.7193
Epoch 6/50
12s - loss:  0.1319 - binary_crossentropy:  0.1319 - auc:  0.7259 - val_binary_crossentropy:  0.1306 - val_auc:  0.7181
Epoch 7/50
12s - loss:  0.1313 - binary_crossentropy:  0.1313 - auc:  0.7308 - val_binary_c

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 256), dnn_dropout=0.0,
               dnn_activation='swish', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


es = EarlyStopping(monitor='val_auc', min_delta=0, verbose=1, patience=5, mode='min')
# mdckpt = ModelCheckpoint(filepath='김동영/model.ckpt', monitor='val_binary_crossentropy', verbose=1, save_best_only=True, mode='min')

# 모델 학습
hist = model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=50, verbose=2, validation_split=0.2, callbacks=[es])

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/50
13s - loss:  0.1580 - binary_crossentropy:  0.1550 - auc:  0.6683 - val_binary_crossentropy:  0.1326 - val_auc:  0.6874
Epoch 2/50
12s - loss:  0.1349 - binary_crossentropy:  0.1349 - auc:  0.6963 - val_binary_crossentropy:  0.1322 - val_auc:  0.6953


KeyboardInterrupt: ignored

In [ ]:
hist.model

DeepFM(
  (embedding_dict): ModuleDict(
    (viewer_gender): Embedding(3, 4)
    (content_used): Embedding(2, 4)
    (content_cat_1): Embedding(21, 4)
    (content_cat_2): Embedding(94, 4)
    (content_cat_3): Embedding(186, 4)
    (content_status): Embedding(3, 4)
    (content_delivery_fee): Embedding(3, 4)
    (content_b_pay): Embedding(2, 4)
    (content_place): Embedding(11, 4)
  )
  (linear_model): Linear(
    (embedding_dict): ModuleDict(
      (viewer_gender): Embedding(3, 1)
      (content_used): Embedding(2, 1)
      (content_cat_1): Embedding(21, 1)
      (content_cat_2): Embedding(94, 1)
      (content_cat_3): Embedding(186, 1)
      (content_status): Embedding(3, 1)
      (content_delivery_fee): Embedding(3, 1)
      (content_b_pay): Embedding(2, 1)
      (content_place): Embedding(11, 1)
    )
  )
  (out): PredictionLayer()
  (fm): FM()
  (dnn): DNN(
    (dropout): Dropout(p=0.0, inplace=False)
    (linears): ModuleList(
      (0): Linear(in_features=56, out_features=512, 

# 1227

- 이미지 + onehot 전부

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='relu', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/20
179s - loss:  0.1366 - binary_crossentropy:  0.1366 - auc:  0.6902 - val_binary_crossentropy:  0.1336 - val_auc:  0.7115
Epoch 2/20
177s - loss:  0.1336 - binary_crossentropy:  0.1336 - auc:  0.7072 - val_binary_crossentropy:  0.1337 - val_auc:  0.7135
Epoch 3/20
175s - loss:  0.1325 - binary_crossentropy:  0.1325 - auc:  0.7162 - val_binary_crossentropy:  0.1372 - val_auc:  0.7103
Epoch 4/20
176s - loss:  0.1322 - binary_crossentropy:  0.1322 - auc:  0.7212 - val_binary_crossentropy:  0.1335 - val_auc:  0.7225
Epoch 5/20
175s - loss:  0.1320 - binary_crossentropy:  0.1320 - auc:  0.7230 - val_binary_crossentropy:  0.1349 - val_auc:  0.7209
Epoch 6/20
177s - loss:  0.1317 - binary_crossentropy:  0.1317 - auc:  0.7279 - val_binary_crossentropy:  0.1339 - val_auc:  0.7126
Epoch 7/20
175s - loss:  0.1310 - binary_crossentropy:  0.1310 - auc:  0.7331 - val_b

## 다 포함

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256, 128), dnn_dropout=0.1,
               dnn_activation='relu', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/20
27s - loss:  0.1407 - binary_crossentropy:  0.1407 - auc:  0.6689 - val_binary_crossentropy:  0.1321 - val_auc:  0.6931
Epoch 2/20
27s - loss:  0.1349 - binary_crossentropy:  0.1349 - auc:  0.6986 - val_binary_crossentropy:  0.1310 - val_auc:  0.7062
Epoch 3/20
27s - loss:  0.1334 - binary_crossentropy:  0.1334 - auc:  0.7131 - val_binary_crossentropy:  0.1309 - val_auc:  0.7114
Epoch 4/20
26s - loss:  0.1328 - binary_crossentropy:  0.1328 - auc:  0.7192 - val_binary_crossentropy:  0.1301 - val_auc:  0.7155
Epoch 5/20
27s - loss:  0.1319 - binary_crossentropy:  0.1319 - auc:  0.7265 - val_binary_crossentropy:  0.1302 - val_auc:  0.7165
Epoch 6/20
26s - loss:  0.1312 - binary_crossentropy:  0.1313 - auc:  0.7307 - val_binary_crossentropy:  0.1295 - val_auc:  0.7226
Epoch 7/20
26s - loss:  0.1308 - binary_crossentropy:  0.1308 - auc:  0.7347 - val_binary_c

test LogLoss 0.1318
test AUC 0.7343
test RIG 0.0689

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 256), dnn_dropout=0.0,
               dnn_activation='relu', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=50, verbose=2, validation_split=0.0)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 588555 samples, validate on 0 samples, 1150 steps per epoch
Epoch 1/50
20s - loss:  0.1366 - binary_crossentropy:  0.1366 - auc:  0.6928
Epoch 2/50
20s - loss:  0.1335 - binary_crossentropy:  0.1335 - auc:  0.7089
Epoch 3/50
20s - loss:  0.1330 - binary_crossentropy:  0.1330 - auc:  0.7153
Epoch 4/50
20s - loss:  0.1328 - binary_crossentropy:  0.1328 - auc:  0.7183
Epoch 5/50
20s - loss:  0.1325 - binary_crossentropy:  0.1325 - auc:  0.7197
Epoch 6/50
20s - loss:  0.1322 - binary_crossentropy:  0.1322 - auc:  0.7224
Epoch 7/50
20s - loss:  0.1321 - binary_crossentropy:  0.1321 - auc:  0.7254
Epoch 8/50
20s - loss:  0.1317 - binary_crossentropy:  0.1317 - auc:  0.7282
Epoch 9/50
20s - loss:  0.1316 - binary_crossentropy:  0.1315 - auc:  0.7284
Epoch 10/50
20s - loss:  0.1314 - binary_crossentropy:  0.1314 - auc:  0.7311
Epoch 11/50
20s - loss:  0.1312 - binary_crossentropy:  0.1312 - auc:  0.7334
Epoch 12/50
20s - loss:  0.1310 

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256), dnn_dropout=0.0,
               dnn_activation='relu', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/20
21s - loss:  0.1443 - binary_crossentropy:  0.1443 - auc:  0.6785 - val_binary_crossentropy:  0.1313 - val_auc:  0.6998
Epoch 2/20
21s - loss:  0.1348 - binary_crossentropy:  0.1347 - auc:  0.7011 - val_binary_crossentropy:  0.1301 - val_auc:  0.7132
Epoch 3/20
21s - loss:  0.1340 - binary_crossentropy:  0.1340 - auc:  0.7096 - val_binary_crossentropy:  0.1313 - val_auc:  0.7112
Epoch 4/20
21s - loss:  0.1336 - binary_crossentropy:  0.1336 - auc:  0.7122 - val_binary_crossentropy:  0.1313 - val_auc:  0.7041
Epoch 5/20
21s - loss:  0.1334 - binary_crossentropy:  0.1334 - auc:  0.7127 - val_binary_crossentropy:  0.1306 - val_auc:  0.7077
Epoch 6/20
21s - loss:  0.1331 - binary_crossentropy:  0.1331 - auc:  0.7173 - val_binary_crossentropy:  0.1320 - val_auc:  0.7037
Epoch 7/20
21s - loss:  0.1330 - binary_crossentropy:  0.1330 - auc:  0.7185 - val_binary_c

In [ ]:
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
# model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
#                 task='binary',
#                 l2_reg_embedding=1e-5, device=device)

model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(512, 256, 128), dnn_dropout=0.0,
               dnn_activation='relu', dnn_use_bn=True, task='binary', device=device, gpus=[0])


# optimizer, loss 설정
model.compile("adam", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )


# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=512, epochs=20, verbose=2, validation_split=0.2)

# 평가
pred_ans = model.predict(test_model_input, 512)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
parallel running on these gpus: [0]
Train on 470844 samples, validate on 117711 samples, 920 steps per epoch
Epoch 1/20
21s - loss:  0.1417 - binary_crossentropy:  0.1417 - auc:  0.6719 - val_binary_crossentropy:  0.1321 - val_auc:  0.6953
Epoch 2/20
19s - loss:  0.1345 - binary_crossentropy:  0.1345 - auc:  0.7009 - val_binary_crossentropy:  0.1312 - val_auc:  0.7059
Epoch 3/20
19s - loss:  0.1332 - binary_crossentropy:  0.1332 - auc:  0.7135 - val_binary_crossentropy:  0.1314 - val_auc:  0.7166
Epoch 4/20
19s - loss:  0.1325 - binary_crossentropy:  0.1325 - auc:  0.7191 - val_binary_crossentropy:  0.1307 - val_auc:  0.7129
Epoch 5/20
19s - loss:  0.1318 - binary_crossentropy:  0.1318 - auc:  0.7265 - val_binary_crossentropy:  0.1297 - val_auc:  0.7219
Epoch 6/20
19s - loss:  0.1314 - binary_crossentropy:  0.1313 - auc:  0.7295 - val_binary_crossentropy:  0.1292 - val_auc:  0.7253
Epoch 7/20
19s - loss:  0.1306 - binary_crossentropy:  0.1306 - auc:  0.7367 - val_binary_c

# 학습 - 1223

- Standard Scaling, 이미지 벡터 제외

In [ ]:
# 4.Define Model,train,predict and evaluate
# cpu gpu 설정
# device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

# 모델 정의
model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                task='binary',
                l2_reg_embedding=1e-5, device=device)

# optimizer, loss 설정
model.compile("adagrad", "binary_crossentropy",
                metrics=["binary_crossentropy", "auc"], )
# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=1024, epochs=20, verbose=2, validation_split=0.0)

# 예측
pred_ans = model.predict(test_model_input, 1024)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda ready...
cuda:0
Train on 588555 samples, validate on 0 samples, 575 steps per epoch
Epoch 1/20
8s - loss:  0.1388 - binary_crossentropy:  0.1388 - auc:  0.6745
Epoch 2/20
8s - loss:  0.1350 - binary_crossentropy:  0.1350 - auc:  0.6879
Epoch 3/20
8s - loss:  0.1348 - binary_crossentropy:  0.1348 - auc:  0.6896
Epoch 4/20
8s - loss:  0.1346 - binary_crossentropy:  0.1346 - auc:  0.6899
Epoch 5/20
8s - loss:  0.1346 - binary_crossentropy:  0.1346 - auc:  0.6918
Epoch 6/20
8s - loss:  0.1345 - binary_crossentropy:  0.1344 - auc:  0.6924
Epoch 7/20
8s - loss:  0.1344 - binary_crossentropy:  0.1344 - auc:  0.6931
Epoch 8/20
8s - loss:  0.1343 - binary_crossentropy:  0.1343 - auc:  0.6942
Epoch 9/20
8s - loss:  0.1342 - binary_crossentropy:  0.1342 - auc:  0.6941
Epoch 10/20
8s - loss:  0.1342 - binary_crossentropy:  0.1342 - auc:  0.6956
Epoch 11/20
8s - loss:  0.1341 - binary_crossentropy:  0.1341 - auc:  0.6955
Epoch 12/20
8s - loss:  0.1341 - binary_crossentropy:  0.1341 - auc:  0.6

- Standard Scaling, 이미지 벡터 전부 사용

In [ ]:
# 모델 학습
model.fit(x = train_model_input, y = train[target].values, batch_size=1024, epochs=20, verbose=2, validation_split=0.0)

# 예측
pred_ans = model.predict(test_model_input, 1024)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))
print("test RIG", round(get_rig(train[target].values, test[target].values, pred_ans), 4))

cuda:0
Train on 588555 samples, validate on 0 samples, 575 steps per epoch
Epoch 1/20
11s - loss:  0.1303 - binary_crossentropy:  0.1303 - auc:  0.7360
Epoch 2/20
11s - loss:  0.1302 - binary_crossentropy:  0.1302 - auc:  0.7356
Epoch 3/20
11s - loss:  0.1302 - binary_crossentropy:  0.1302 - auc:  0.7365
Epoch 4/20
11s - loss:  0.1302 - binary_crossentropy:  0.1302 - auc:  0.7366
Epoch 5/20
11s - loss:  0.1302 - binary_crossentropy:  0.1302 - auc:  0.7366
Epoch 6/20
11s - loss:  0.1301 - binary_crossentropy:  0.1301 - auc:  0.7376
Epoch 7/20
11s - loss:  0.1301 - binary_crossentropy:  0.1301 - auc:  0.7374
Epoch 8/20
11s - loss:  0.1300 - binary_crossentropy:  0.1300 - auc:  0.7381
Epoch 9/20
11s - loss:  0.1300 - binary_crossentropy:  0.1300 - auc:  0.7387
Epoch 10/20
11s - loss:  0.1300 - binary_crossentropy:  0.1300 - auc:  0.7386
Epoch 11/20
11s - loss:  0.1299 - binary_crossentropy:  0.1299 - auc:  0.7391
Epoch 12/20
11s - loss:  0.1299 - binary_crossentropy:  0.1299 - auc:  0.739